In [1]:
import pandas as pd
from prepare import basic_clean, tokenize, stem, lemmatize, remove_stopwords, prep_article_data, words
from acquire import scrape_github_data

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

# Import Decision Tree and Random Forest ;D
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

import warnings
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
df = pd.read_json('data.json')

df = words(df)

#df.info()
#df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   repo                      100 non-null    object
 1   language                  100 non-null    object
 2   readme                    100 non-null    object
 3   clean                     100 non-null    object
 4   stemmed                   100 non-null    object
 5   lemmatized                100 non-null    object
 6   contains_python_keywords  100 non-null    int64 
 7   contains_cpp_keywords     100 non-null    int64 
 8   contains_js_keywords      100 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 7.2+ KB


,repo,language,readme,clean,stemmed,lemmatized,contains_python_keywords,contains_cpp_keywords,contains_js_keywords
0,bitcoin/bitcoin,C++,Bitcoin Core integration/staging tree\n=======...,bitcoin core integrationstaging tree httpsbitc...,bitcoin core integrationstag tree httpsbitcoin...,bitcoin core integrationstaging tree httpsbitc...,1,1,0
1,bitcoinbook/bitcoinbook,other,Code Examples: ![travis_ci](https://travis-ci....,code examples traviscihttpstravisciorgbitcoinb...,code exampl traviscihttpstravisciorgbitcoinboo...,code example traviscihttpstravisciorgbitcoinbo...,0,0,0
2,bitcoin/bips,other,"People wishing to submit BIPs, first should pr...",people wishing submit bips first propose idea ...,peopl wish submit bip first propos idea docume...,people wishing submit bips first propose idea ...,1,1,0
3,bitcoinjs/bitcoinjs-lib,other,# BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...,bitcoinjs bitcoinjslib github cihttpsgithubcom...,bitcoinj bitcoinjslib github cihttpsgithubcomb...,bitcoinjs bitcoinjslib github cihttpsgithubcom...,1,0,1
4,spesmilo/electrum,Python,Electrum - Lightweight Bitcoin client\n=======...,electrum lightweight bitcoin client licence mi...,electrum lightweight bitcoin client licenc mit...,electrum lightweight bitcoin client licence mi...,1,1,0


### Modeling prep

In [3]:
# Make copies of df with prepared columns and target
clean_df = df.copy()[['language', 'clean']]
stem_df = df.copy()[['language', 'stemmed']]
lem_df = df.copy()[['language', 'lemmatized']]


# Get splits for each of the above dfs and isolate target
X_clean = clean_df[['clean']]
y_clean = clean_df.language

X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean, y_clean, test_size=.2, random_state=302)
X_clean_train, X_clean_validate, y_clean_train, y_clean_validate  = train_test_split(X_clean_train, y_clean_train, test_size=.3, random_state=302)

print(X_clean_train.shape, X_clean_validate.shape, X_clean_test.shape)

X_stem = stem_df[['stemmed']]
y_stem = stem_df.language

X_stem_train, X_stem_test, y_stem_train, y_stem_test = train_test_split(X_stem, y_stem, test_size=.2, random_state=302)
X_stem_train, X_stem_validate, y_stem_train, y_stem_validate  = train_test_split(X_stem_train, y_stem_train, test_size=.3, random_state=302)

print(X_stem_train.shape, X_stem_validate.shape, X_stem_test.shape)

X_lem = lem_df[['lemmatized']]
y_lem = lem_df.language

X_lem_train, X_lem_test, y_lem_train, y_lem_test = train_test_split(X_lem, y_lem, test_size=.2, random_state=302)
X_lem_train, X_lem_validate, y_lem_train, y_lem_validate  = train_test_split(X_lem_train, y_lem_train, test_size=.3, random_state=302)

print(X_lem_train.shape, X_lem_validate.shape, X_lem_test.shape)

(56, 1) (24, 1) (20, 1)
(56, 1) (24, 1) (20, 1)
(56, 1) (24, 1) (20, 1)


### Models

In [14]:
# Make vectorizer objects for bags of words (clean_df)
cv_clean = CountVectorizer()
tfidf_clean = TfidfVectorizer()

#Bags of words
cv_clean_bow = cv_clean.fit_transform(X_clean_train.clean)
tf_clean_bow = tfidf_clean.fit_transform(X_clean_train.clean)

In [15]:
# Make and fit decision tree object for cv_clean_bow
cv_tree1 = DecisionTreeClassifier(max_depth=5)
cv_tree1.fit(cv_clean_bow, y_clean_train)

#Output tree score
print(f'CV tree 1 score: {cv_tree1.score(cv_clean_bow, y_clean_train)}')

#Make and fit decision tree object for tf_clean_bow
tf_tree1 = DecisionTreeClassifier(max_depth=5)
tf_tree1.fit(tf_clean_bow, y_clean_train)

#Output tree score
print(f'TF IDF tree 1 score: {tf_tree1.score(tf_clean_bow, y_clean_train)}')

CV tree 1 score: 0.8571428571428571
TF IDF tree 1 score: 0.9107142857142857


In [16]:
# "Stemmed" models
cv_stem = CountVectorizer()
tfidf_stem = TfidfVectorizer()

# Bags
cv_stem_bow = cv_stem.fit_transform(X_stem_train.stemmed)
tf_stem_bow = tfidf_stem.fit_transform(X_stem_train.stemmed)

# Make and fit decision tree object for cv_stem_bow
cv_tree2 = DecisionTreeClassifier(max_depth=5)
cv_tree2.fit(cv_stem_bow, y_stem_train)

#Output tree score
print(f'CV tree 2 score: {cv_tree2.score(cv_stem_bow, y_stem_train)}')

#Make and fit decision tree object for tf_stem_bow
tf_tree2 = DecisionTreeClassifier(max_depth=5)
tf_tree2.fit(tf_stem_bow, y_stem_train)

#Output tree score
print(f'TF IDF tree 2 score: {tf_tree2.score(tf_stem_bow, y_stem_train)}')

CV tree 2 score: 0.8571428571428571
TF IDF tree 2 score: 0.9107142857142857


In [25]:
# "Lemmatized" models
cv_lem = CountVectorizer()
tfidf_lem = TfidfVectorizer()

cv_lem_bow = cv_lem.fit_transform(X_lem_train.lemmatized)
tf_lem_bow = tfidf_lem.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem_bow
cv_tree3 = DecisionTreeClassifier(max_depth=5)
cv_tree3.fit(cv_lem_bow, y_lem_train)

#Make and fit decision tree object for tf_lem_bow
tf_tree3 = DecisionTreeClassifier(max_depth=5)
tf_tree3.fit(tf_lem_bow, y_lem_train)

# Output tree scores
print(f'CV tree score: {cv_tree3.score(cv_lem_bow, y_lem_train)}') 
print(f'TFIDF tree score: {tf_tree3.score(tf_lem_bow, y_lem_train)}')

CV tree score: 0.8571428571428571
TFIDF tree score: 0.9107142857142857


In [18]:
dec_tree_training_scores= {
    'CV_clean': cv_tree1.score(cv_clean_bow, y_clean_train),
    'CV_stem': cv_tree2.score(cv_stem_bow, y_stem_train),
    'CV_lem': cv_tree3.score(cv_lem_bow, y_lem_train),
    'TFIDF_clean': tf_tree1.score(tf_clean_bow, y_clean_train),
    'TFIDF_stem': tf_tree2.score(tf_stem_bow, y_stem_train),
    'TFIDF_lem': tf_tree3.score(tf_lem_bow, y_lem_train)
}

dec_tree_training_scores

{'CV_clean': 0.8571428571428571,
 'CV_stem': 0.8571428571428571,
 'CV_lem': 0.8571428571428571,
 'TFIDF_clean': 0.9107142857142857,
 'TFIDF_stem': 0.9107142857142857,
 'TFIDF_lem': 0.9107142857142857}

### Takeaways: Both vectorizers scoring the same on clean, stemmed, and lemmatized versions of README text

### Validate

In [20]:
#Transform validate split with vectorizer
tf_clean_bow_val = tfidf_clean.transform(X_clean_validate.clean)

#Get tf_tree1 score on validate
tf_tree1.score(tf_clean_bow_val, y_clean_validate)

0.625

### Considerable drop off. Probably overfit.

In [22]:
#Transform validate split with vectorizer
tf_stem_bow_val = tfidf_stem.transform(X_stem_validate.stemmed)

#Get tf_tree1 score on validate
tf_tree2.score(tf_stem_bow_val, y_stem_validate)

0.5416666666666666

### Even worse dropoff...

In [26]:
#Transform validate split with vectorizer
tf_lem_bow_val = tfidf_lem.transform(X_lem_validate.lemmatized)

#Get tf_tree1 score on validate
tf_tree3.score(tf_lem_bow_val, y_lem_validate)

0.5416666666666666

### Lemmatized and stemmed preparations scored the same: 54.2%  Cleaned preparation scored 62.5% and will be tested.

In [27]:
tf_clean_bow_test = tfidf_clean.transform(X_clean_test.clean)

tf_tree1.score(tf_clean_bow_test, y_clean_test)

0.35

# Results: Decision Tree model with a max depth of 5 and TFIDF Vectorizer, using "clean" version of README text, performs at 35% accuracy on unseen data.

# Conclusions: Different types of models and adjustments to hyperparameters may yield better results.